# 🏠 사회취약지수 분석 및 지도 시각화

이 노트북에서는 한국의 지역별 사회취약지수를 분석하고 지도로 시각화합니다.

## 📋 분석 목표
- 자연재해 위험도와 노후주택 현황을 종합한 사회취약지수 계산
- 지역별 취약성 비교 및 시각화
- 인터랙티브 지도 생성

## 🎯 분석 지표
1. **전체 위험도** (40%): 자연재해 위험도 종합 지표
2. **고위험도** (30%): 높은 위험 수준의 재해 위험도
3. **노후주택비율** (30%): 노후 건축물 비율

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins
import warnings
from pathlib import Path

warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False

# 시각화 스타일 설정
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("라이브러리 임포트 완료!")

## 📁 데이터 로드 및 전처리

In [ ]:
# 데이터 로드
df = pd.read_csv('../data/processed/processed_data.csv')
print("데이터 형태:", df.shape)
print("\n데이터 미리보기:")
df.head()

In [ ]:
# 데이터 정보 확인
print("데이터 정보:")
print(df.info())
print("\n기술통계:")
print(df.describe())

In [ ]:
# 결측치 확인 및 처리
print("결측치 확인:")
print(df.isnull().sum())

# 대전광역시의 low_risk 결측치를 0으로 처리
df['low_risk'] = df['low_risk'].fillna(0)
print("\n결측치 처리 후:")
print(df.isnull().sum())

## 📊 사회취약지수 계산

In [ ]:
# 사회취약지수 계산
# 위험도와 노후주택비율을 고려한 종합 취약지수
df['vulnerability_index'] = (
    df['total_risk'] * 0.4 +  # 전체 위험도 (40%)
    df['high_risk'] * 0.3 +   # 고위험도 (30%)
    df['aged_housing_ratio'] * 0.3  # 노후주택비율 (30%)
)

# 취약지수 정규화 (0-100 스케일)
df['vulnerability_normalized'] = (
    (df['vulnerability_index'] - df['vulnerability_index'].min()) / 
    (df['vulnerability_index'].max() - df['vulnerability_index'].min()) * 100
)

print("사회취약지수 계산 완료")
print("\n취약지수 상위 5개 지역:")
print(df.nlargest(5, 'vulnerability_normalized')[['region', 'vulnerability_normalized', 'total_risk', 'aged_housing_ratio']])

## 📈 데이터 시각화

In [ ]:
# 시각화를 위한 그래프 생성
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. 취약지수 분포
axes[0, 0].hist(df['vulnerability_normalized'], bins=30, alpha=0.7, color='skyblue')
axes[0, 0].set_title('사회취약지수 분포')
axes[0, 0].set_xlabel('취약지수')
axes[0, 0].set_ylabel('빈도')

# 2. 위험도 vs 노후주택비율 산점도
axes[0, 1].scatter(df['total_risk'], df['aged_housing_ratio'], 
                   c=df['vulnerability_normalized'], cmap='viridis', alpha=0.6)
axes[0, 1].set_title('위험도 vs 노후주택비율')
axes[0, 1].set_xlabel('전체 위험도')
axes[0, 1].set_ylabel('노후주택비율 (%)')
plt.colorbar(axes[0, 1].collections[0], ax=axes[0, 1])

# 3. 상위 10개 취약 지역
top_vulnerable = df.nlargest(10, 'vulnerability_normalized')
axes[1, 0].barh(range(len(top_vulnerable)), top_vulnerable['vulnerability_normalized'])
axes[1, 0].set_yticks(range(len(top_vulnerable)))
axes[1, 0].set_yticklabels(top_vulnerable['region'], fontsize=8)
axes[1, 0].set_title('상위 10개 취약 지역')
axes[1, 0].set_xlabel('취약지수')

# 4. 지표별 상관관계
correlation_cols = ['total_risk', 'high_risk', 'aged_housing_ratio', 'vulnerability_normalized']
correlation_matrix = df[correlation_cols].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, ax=axes[1, 1])
axes[1, 1].set_title('지표별 상관관계')

plt.tight_layout()
plt.show()

## 🗺️ 인터랙티브 지도 생성

In [ ]:
# 한국 중심 지도 생성
m = folium.Map(
    location=[36.5, 127.5],  # 한국 중심 좌표
    zoom_start=7,
    tiles='OpenStreetMap'
)

# 취약지수에 따른 색상 매핑
def get_color(vulnerability):
    if vulnerability >= 80:
        return 'red'
    elif vulnerability >= 60:
        return 'orange'
    elif vulnerability >= 40:
        return 'yellow'
    else:
        return 'green'

# 각 지역에 마커 추가 (임시 좌표 사용)
for idx, row in df.iterrows():
    # 실제로는 각 지역의 정확한 좌표가 필요합니다
    # 여기서는 임시로 한국 내 랜덤 좌표를 사용합니다
    lat = 33.0 + (row.name % 10) * 0.5
    lon = 124.0 + (row.name % 15) * 0.5
    
    folium.CircleMarker(
        location=[lat, lon],
        radius=8,
        popup=f"{row['region']}<br>취약지수: {row['vulnerability_normalized']:.1f}<br>위험도: {row['total_risk']:.2f}<br>노후주택비율: {row['aged_housing_ratio']:.1f}%",
        color=get_color(row['vulnerability_normalized']),
        fill=True,
        fillOpacity=0.7
    ).add_to(m)

# 범례 추가
legend_html = '''
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 150px; height: 90px; 
            background-color: white; border:2px solid grey; z-index:9999; 
            font-size:14px; ">
            &nbsp; <b>사회취약지수</b> <br>
            &nbsp; <i class="fa fa-circle fa-1x" style="color:red"></i> 매우 높음 (80+) <br>
            &nbsp; <i class="fa fa-circle fa-1x" style="color:orange"></i> 높음 (60-79) <br>
            &nbsp; <i class="fa fa-circle fa-1x" style="color:yellow"></i> 보통 (40-59) <br>
            &nbsp; <i class="fa fa-circle fa-1x" style="color:green"></i> 낮음 (0-39) <br>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

m

## 📊 지역별 분석 결과

In [ ]:
# 지역별 요약 통계
print("=== 지역별 사회취약지수 요약 ===")
summary_stats = df.groupby('region').agg({
    'vulnerability_normalized': ['mean', 'std', 'min', 'max'],
    'total_risk': 'mean',
    'aged_housing_ratio': 'mean'
}).round(2)

summary_stats.columns = ['평균_취약지수', '표준편차', '최소값', '최대값', '평균_위험도', '평균_노후주택비율']
print(summary_stats.sort_values('평균_취약지수', ascending=False))

# 상위 10개 취약 지역
print("\n=== 상위 10개 취약 지역 ===")
top_vulnerable = df.nlargest(10, 'vulnerability_normalized')[
    ['region', 'vulnerability_normalized', 'total_risk', 'aged_housing_ratio']
]
print(top_vulnerable)

## 💾 결과 저장

In [ ]:
# 분석 결과 저장
output_path = '../results/'
os.makedirs(output_path, exist_ok=True)

# 전체 데이터 저장
df.to_csv(f'{output_path}social_vulnerability_analysis.csv', 
          index=False, encoding='utf-8-sig')

# 지도 저장
m.save(f'{output_path}social_vulnerability_map.html')

print("결과 저장 완료!")
print(f"- 분석 데이터: {output_path}social_vulnerability_analysis.csv")
print(f"- 인터랙티브 지도: {output_path}social_vulnerability_map.html")

## 📋 분석 결과 요약

### 🎯 주요 발견사항
1. **사회취약지수 분포**: [분포 특성] 분포를 보이며, 평균 [평균값]점
2. **취약 지역**: [상위 지역]이 가장 높은 취약지수를 보임
3. **지표 상관관계**: [주요 상관관계 발견사항]

### 🚨 우선 대응 필요 지역
극한 강우 대응을 위해 다음 지역들의 사회취약도 개선이 시급:
- [취약 지역 1]: 취약지수 [값]점
- [취약 지역 2]: 취약지수 [값]점
- [취약 지역 3]: 취약지수 [값]점

### 📊 다음 단계
1. 하수도 인프라 지수와 결합한 통합 위험도 분석
2. 극한 강우 데이터와의 상관관계 분석
3. 지역별 우선순위 매트릭스 구축